In [1]:
import json
import yaml
import uuid

In [7]:
import wombat.bsub as bsub

In [2]:
t = yaml.safe_load(open('/home/estorrs/pecgs-pipeline/cwl/charger/template.charger.yaml'))
t

{'vcf': {'class': 'File', 'path': 'a/file/path'}}

In [3]:
template = {
    'vcf': {
        'class': 'File',
        'path': '/scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/pecgs_TN_wxs_bam/runs/C3L-00677_2a9edf1e-76df-4595-9125-fb9b9d919621/cromwell-workdir/cromwell-executions/pecgs_TN_wxs_bam.cwl/05158e98-46ce-4fbe-84d5-e583fd353767/call-run_tinjasmine/TinJasmine.cwl/0dbce154-310b-4c81-a820-be6e67ab7542/call-canonical_filter/execution/output/HotspotFiltered.vcf'
    }
}
template

{'vcf': {'class': 'File',
  'path': '/scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/pecgs_TN_wxs_bam/runs/C3L-00677_2a9edf1e-76df-4595-9125-fb9b9d919621/cromwell-workdir/cromwell-executions/pecgs_TN_wxs_bam.cwl/05158e98-46ce-4fbe-84d5-e583fd353767/call-run_tinjasmine/TinJasmine.cwl/0dbce154-310b-4c81-a820-be6e67ab7542/call-canonical_filter/execution/output/HotspotFiltered.vcf'}}

In [5]:
yaml.safe_dump(template, open('../tests/data/charger/C3L-00677_inputs.yaml', 'w'))

In [8]:
workflow_root = '/scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/charger/C3L-00677'
output_fp = '../tests/data/charger/C3L-00677.cromwell-config-db.compute1.dat'
bsub.save_compute1_cromwell_template(workflow_root, output_fp)

In [9]:
TEST_DIR_ROOT = '/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/charger'
TOOL_ROOT = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline'

dconfig = os.path.join(TEST_DIR_ROOT, 'C3L-00677.cromwell-config-db.compute1.dat')
cwl_fp = os.path.join(TOOL_ROOT, 'cwl', 'charger', 'charger.cwl')
inputs_fp = os.path.join(TEST_DIR_ROOT, 'C3L-00677_inputs.yaml')
volumes = [TEST_DIR_ROOT, TOOL_ROOT, workflow_root, '/storage1/fs1/dinglab/Active', '/storage1/fs1/m.wyczalkowski/Active', '/scratch1/fs1/dinglab/estorrs']

NameError: name 'os' is not defined

In [38]:
args = bsub.DEFAULT_ARGS
args['group_name'] = 'test_cromwell'
args

{'mem': 10,
 'n_processes': 1,
 'max_mem': None,
 'docker': 'python:3.8',
 'queue': 'dinglab',
 'group': 'compute-dinglab',
 'group_name': 'test_cromwell',
 'n_concurrent': 10,
 'interactive': False,
 'username': 'estorrs'}

In [39]:
start_commands, run_command = bsub.cromwell_commands(dconfig, cwl_fp, inputs_fp, args, volumes, workflow_root=workflow_root)
filepath = os.path.join('../tests/data/align_wxs/start_server.sh')
bsub.write_command_file(start_commands, filepath)
filepath = os.path.join('../tests/data/align_wxs/run_job.sh')
bsub.write_command_file([run_command], filepath)

In [40]:
for c in start_commands: print(c)

mkdir -p /scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/align_wxs/HT191P1-S1H1A3Y3/cromwell-workdir/logs
source /opt/ibm/lsfsuite/lsf/conf/lsf.conf
export LSF_DOCKER_NETWORK=host
export LSF_DOCKER_VOLUMES="/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/align_wxs:/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/align_wxs /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline:/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline /scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/align_wxs/HT191P1-S1H1A3Y3:/scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/align_wxs/HT191P1-S1H1A3Y3 /storage1/fs1/dinglab/Active:/storage1/fs1/dinglab/Active /storage1/fs1/m.wyczalkowski/Active:/storage1/fs1/m.wyczalkowski/Active /scratch1/fs1/dinglab/estorrs:/scratch1/fs1/dinglab/estorrs"
bgadd -L 10 /estorrs/test_cromwell
bsub -n 1 -q dinglab-interactive -G compute-dinglab -a 'docker(mwyczalkowski/cromwell-runner)' -g /estorrs/test_cromw

In [41]:
print(run_command)

/usr/bin/java -Dconfig.file=/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/align_wxs/HT191P1-S1H1A3Y3.cromwell-config-db.compute1.dat -Djavax.net.ssl.trustStorePassword=changeit -Djavax.net.ssl.trustStore=/gscmnt/gc2560/core/genome/cromwell/cromwell.truststore -jar /usr/local/cromwell/cromwell-47.jar run -t cwl -i /storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/align_wxs/HT191P1-S1H1A3Y3_inputs.yaml /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline/cwl/align_dnaseq/align_dnaseq_workflow.cwl
